# Stocks Price Prediction With GPR

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, DotProduct, RBF, Matern, RationalQuadratic, ExpSineSquared, Kernel, Hyperparameter
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import quandl
from sklearn import preprocessing
np.random.seed(seed=0)  # Set seed for NumPy
random_state = 0

## Config your hyperparameters here.

In [ ]:
stock_id = 'MULTPL/SP500_REAL_PRICE_YEAR' # the stock you want to get
input_data_length = 38 # length of input data

## Get Data

In [ ]:
# Get data via Quandl API
quandl.ApiConfig.api_key = 'No6kZxhas7xsxr5_mB-f'
origial_data = quandl.get(stock_id)

In [ ]:
data = origial_data[113:165]

## Preprocess Data
Note: the normalization will remarkably eaze the pain of tuning.

In [ ]:
value = np.array(data.Value).reshape(-1, 1)
scaler = preprocessing.StandardScaler().fit(value)
value = scaler.transform(value)
y = value[:input_data_length]
x = np.arange(input_data_length).reshape(-1,1)

## Fit GPR model and Predict

In [ ]:
from sklearn.utils.optimize import _check_optimize_result
import scipy
class MyGPR(GaussianProcessRegressor):
    def __init__(self, *args, max_iter=2e05, gtol=1e-06, **kwargs):
        super().__init__(*args, **kwargs)
        self._max_iter = max_iter
        self._gtol = gtol

    def _constrained_optimization(self, obj_func, initial_theta, bounds):
        if self.optimizer == "fmin_l_bfgs_b":
            opt_res = scipy.optimize.minimize(obj_func, initial_theta, method="L-BFGS-B", jac=True, bounds=bounds, options={'maxiter':self._max_iter, 'gtol': self._gtol})
            _check_optimize_result("lbfgs", opt_res)
            theta_opt, func_min = opt_res.x, opt_res.fun
        elif callable(self.optimizer):
            theta_opt, func_min = self.optimizer(obj_func, initial_theta, bounds=bounds)
        else:
            raise ValueError("Unknown optimizer %s." % self.optimizer)
        return theta_opt, func_min

In [ ]:
# [Important] choice of kernel
kernel = (
    # long-term rising
    DotProduct(sigma_0=1)
    # # flunctuate in a year
    # + 0.1*RBF(length_scale=20) * ExpSineSquared(length_scale=0.1 , periodicity=3)
    + ExpSineSquared(length_scale=0.1 , periodicity=15)
    # + 0.1*RBF(length_scale=5) * ExpSineSquared(length_scale=0.1 , periodicity=50)
    # # minor changes in a year
    # + 0.1*RationalQuadratic(alpha=1, length_scale=1) + 1 * RBF(length_scale=0.01)
    # # white noise
    + WhiteKernel(noise_level=0.01)
)
gpr = MyGPR(kernel=kernel, random_state=random_state, n_restarts_optimizer=8)
x_new = np.arange(0, len(data)).reshape(-1,1)
y_hat_old, y_sigma_old = gpr.predict(x_new, return_std=True)
gpr.fit(x, y)
y_hat_new, y_sigma_new = gpr.predict(x_new, return_std=True)

## Plot Data

In [ ]:
sns.set(style="darkgrid", palette="muted", color_codes=True)
# Initialize plot
f, ax = plt.subplots(2, 1, figsize=(10, 10))

# before optimize
# plot predict data
y_hat_old = np.squeeze(y_hat_old)
ax[0].plot(x_new, y_hat_old, 'b')
lower = y_hat_old - y_sigma_old
upper = y_hat_old + y_sigma_old
ax[0].fill_between(x_new.flatten(), lower, upper, alpha=0.3)
# plot original data
ax[0].plot(np.arange(len(value)), value, 'r')
ax[0].set_title(
    "Initial: %s\n"
    % (kernel)
)
ax[0].axvline(x=input_data_length)

# after optimize
y_hat_new = np.squeeze(y_hat_new)
ax[1].plot(x_new, y_hat_new, 'b')
lower = y_hat_new - y_sigma_new
upper = y_hat_new + y_sigma_new
ax[1].fill_between(x_new.flatten(), lower, upper, alpha=0.3)
# plot original data
ax[1].plot(np.arange(len(value)), value, 'r')
# plot original data
ax[1].plot(np.arange(len(value)), value, 'r')
ax[1].set_title(
    "Optimum: %s\nLog-Marginal-Likelihood: %s"
    % (gpr.kernel_, gpr.log_marginal_likelihood(gpr.kernel_.theta))
)
ax[1].axvline(x=input_data_length)
# ax[1].plot(np.arange(len(data.Value)), data.Value, 'g')
plt.show()